In [4]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import json
import pandas as pd
import requests

In [5]:
%load_ext autoreload
%autoreload 2

In [6]:
sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials())

In [53]:
result = sp.search(q="04boE4u1AupbrGlI62WvoO")
print(result['tracks']['items'])

[]


In [98]:
features_list = ['artist', 'name', 'id', 'popularity', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature']
metadata_list = features_list[:4]
analysis_list = features_list[4:]

features = {feature : [] for feature in features_list}
metadata = {feature : features[feature] for feature in metadata_list}
analysis = {feature : features[feature] for feature in analysis_list}

start_year = 2015
end_year = 2019
search_limit = 50

for year in range(start_year, end_year + 1):
    for i in range(0, 2000, search_limit):
        print("\r", "Year {}, iteration {}".format(year, i // search_limit), end="")
        current_tracks = []
        try:
            results = sp.search(q='year:'+str(year), limit=search_limit, type='track', offset=i)
        except (requests.HTTPError, spotipy.SpotifyException):
            pass
        for j, t in enumerate(results['tracks']['items']):
            # Apparently 3 hours of white noise was a popular track in 2016
            # There is no sound analysis for this
            # We have to exclude the track so we don't get an error
            if t['id'] != '04boE4u1AupbrGlI62WvoO':
                current_tracks.append(t['id'])
                for k, v in metadata.items():
                    if k == 'artist':
                        v.append(t['artists'][0]['name'])
                    else:
                        v.append(t[k])
        try:
            feature_results = sp.audio_features(current_tracks)
        except (requests.HTTPError, spotipy.SpotifyException):
            pass
        for feature in feature_results:
            for k, v in analysis.items():
                v.append(feature[k])

 Year 2019, iteration 39

In [102]:
df = pd.DataFrame(features)
print(df.shape)
df.head()

(9999, 17)


,artist,name,id,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,BØRNS,Electric Love,2GiJYvgVaD2HtM8GqD9EgQ,83,0.611,0.797,6,-7.627,0,0.0533,0.00543,0.001370,0.2600,0.518,120.041,218107,4
1,Melanie Martinez,Play Date,4DpNNXFMMxQEKl7r0ykkWA,89,0.680,0.729,5,-5.077,1,0.0475,0.61200,0.000000,0.2240,0.446,123.970,179867,4
2,Tame Impala,The Less I Know The Better,6K4t31amVTZDgR3sKmwUJJ,85,0.640,0.740,4,-4.083,1,0.0284,0.01150,0.006780,0.1670,0.785,116.879,216320,4
3,WILLOW,Wait a Minute!,0y60itmpH0aPKsFiGxmtnh,84,0.764,0.705,3,-5.279,0,0.0278,0.03710,0.000019,0.0943,0.672,101.003,196520,4
4,Chris Stapleton,Tennessee Whiskey,3fqwjXwUGN6vbzIwvyFMhx,79,0.392,0.370,9,-10.888,1,0.0298,0.20500,0.009600,0.0821,0.512,48.718,293293,4


In [103]:
print(min(features['popularity']))

48
